<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Train ML models</h1>
<h3 align="right">Written by the KSO Team</h3>

This notebook takes you through the process of importing a baseline model, training it on a dataset and evaluating the quality of the model. If you do not have a project with us yet, you can run the template project to get a taste of how it all works. 

🔴 <span style="color:red">&nbsp;NOTE: In order to run this notebook, you need to have a Weights and Biases account. If you want to become a member of our Koster team on Weights and Biases, you may request this access by contacting jurie.germishuys@combine.se. But this is not necessary to run the template project. </span>

# 1. Set up and requirements

### Install all the requirements

In [ ]:
from IPython.display import clear_output
import os
import sys

try:
    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/yolov5/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Navigate to the correct folder
    os.chdir("kso/tutorials")

except:
    IN_COLAB = False


# Ensure widgets are shown properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

clear_output()
if IN_COLAB == True:
    print("Running in Colab: All packages are installed and ready to go!")
else:
    print("Running locally... you're good to go!")

In [ ]:
# Replace nearest neighbours script with custom version (due to relative path issue)
import shutil

shutil.copy(
    "../src/multi_tracker_zoo.py",
    "../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py",
)

### Import python packages

In [ ]:
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")

        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
from ipyfilechooser import FileChooser
import kso_utils.tutorials_utils as t_utils
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils
import kso_utils.yolo_utils as y_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

# 2. Train the model

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initiate pp
pp = ProjectProcessor(project)

In [ ]:
# Initiate mlp
mlp = MLProjectProcessor(pp)

In [ ]:
# Only for Template Project (downloading prepared data)
s_utils.get_ml_data(project)

### Configure data paths

If you are running the Template project, the output_folder that you want to select is the ml-template-data. The path to this folder is printed in the cell above. For any other project, it is the folder where you have saved your data.

In [ ]:
# Specify path containing the images and labels folders.
mlp.output_path = kso_widgets.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".", "output"
)

🔴 <span style="color:red">&nbsp;NOTE: Each model type requires a specific folder structure to be in place. To be able to train your own Object Detection models, your data_path must contain a yml file for data and hyperparameters. See https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#11-create-datasetyaml. For image classification models, there should be 3 folders (train, val, test) each containing images in class_name folders. For segmentation models, polygon coordinates are also required. </span>

In [ ]:
# Fix important paths
mlp.setup_paths()

### Choose a suitable experiment name

In [ ]:
exp_name = t_utils.choose_experiment_name()

### Choose model to use for training

In the next cell you will specify the folder (can be any folder of choice) where you want to download the baseline model to, which you will select in the cell after. This baseline model will be used as the starting point for the training.

In [ ]:
# Specify path to download baseline model
download_folder = kso_widgets.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".",
    "model download",
)

In [ ]:
weights = y_utils.choose_baseline_model(download_folder.selected)

### Train model with given configuration

The cell below will ask you which batch size and how many epochs you want to use during training. There are no strict rules for this and the best settings will depend on the choice of GPU and some randomness that we have encountered while training models. Therefore it will be some trial and error. As a starting point we advice to use a batch size of 8. For smaller datasets, we have experienced that 50-100 epochs has been sufficient to get good performance on the model (metrics that have reached a plateau), but to not overfit to the training set. 

In [ ]:
batch_size, epochs, img_h, img_w = mlp.choose_train_params()

In [ ]:
# Give your WandB username, or team name where you want to sent the runs to.
# If you are part of the koster project, you can keep the default 'koster'.
entity = mlp.choose_entity(alt_name=False)

In [ ]:
mlp.train_yolov5(
    exp_name.value,
    weights.artifact_path,
    epochs=epochs.value,
    batch_size=batch_size.value,
    img_size=(img_h.value, img_w.value),
)

# 3. Evaluate model performance

The model is now done with training. To see the loss, precision, recall and some other parameters per training epoch, click on the link in the previous cell. Here you can see your run in Weights and Biases. To evaluate the resulting model, please run the cells below. These execute the standard evaluation process from YOLO. 

For a biological evaluation of the model, please see Notebook 6.

In [ ]:
conf_thres = t_utils.choose_eval_params()

In [ ]:
# Choose model: The folder you want to select for eval_model is the folder with your experiment_name.
eval_model = FileChooser(mlp.project_name)
display(eval_model)

When you run the cell below, you will get some numbers logged on the screen, and 3 files that are stored in the folder 'your_experiment_name'_val. 

The numbers logged on the screen represent the following:
* The first 7 numbers are the: mean precision, mean recal, mean average precision calculated at IOU threshold 0.5 (map@0.5), the mean average precision calculated at different IOU thresholds of 0.5-0.95 with steps of 0.05 (map@0.5:0.95) and then 3 training losses based on predicting the box, object or class. 
* The array gives the ap@0.5 per class.
* The last 3 numbers are the same as the numbers that are already printed in a line above, where it says 'Speed: … ms per....' 

In [ ]:
# Evaluate YOLO Model on Unseen Test data
mlp.eval_yolov5(exp_name.value, eval_model.selected, conf_thres.value)

# (Optional) : 4. Enhance annotations using trained model

Enhancement uses the trained model to increase the amount of annotations in the training data. This should only be done in cases where it is absolutely necessary as bad predictions lead to worse predictions when used to train the next iteration of the model. 


🔴 <span style="color:red">&nbsp;NOTE: We recommend using a relatively high confidence threshold when enhancing trained models as low confidence predictions could significantly impact the quality of your annotated data. This is currently only available for object detection models.  </span>

In [ ]:
eh_conf_thres = t_utils.choose_eval_params()

In [ ]:
mlp.enhance_yolov5(eh_conf_thres.value, img_size=[640, 640])

### Choose run to use as enhanced annotations

In [ ]:
runs = FileChooser(".")
display(runs)

In [ ]:
# Move enhanced annotations to original run folder (NB: This will replace the original annotations)
mlp.enhance_replace(runs.selected)

#### Once you have moved the new labels to the original label location, you can return to Step 2 and train your model again. 

In [ ]:
# END